# Pre-Pre-Processing

Before I start performing any EDA or modeling, I will pre-emptively split my data to have a separate training set of data.

Since the data consists of temporal data (hotel reservations across multiple years), I will split the data into two groups based on two datetime features: Booking Date and the Arrival date.

---

# Consulting ChatGPT

I asked ChatGPT to help me decide if one feature would be better than the other. It's responses are as follows:

## Booking Date

### Pros:
> * *Forecasting Lead Time:* Splitting by booking date allows you to model the lead time between booking and arrival, which is valuable for forecasting future demand well in advance.
> * *Real-world Scenario Modeling:* This approach mirrors real-world scenarios more closely, where predictions are made as bookings occur, regardless of the future arrival date.
> * *Dynamic Pricing and Revenue Management:* Particularly useful for applications in dynamic pricing and revenue management, where understanding booking patterns ahead of time can inform strategic decisions.

### Cons:
> * *Increased Complexity:* The time gap between booking and arrival dates introduces additional complexity, as you need to account for changes over time that could affect the stay (e.g., cancellations, modifications).



## Arrival Date

### Pros:
> * *Cleaner Data Relationships:* Utilizing the arrival date might result in cleaner, more straightforward modeling, as it directly ties the prediction to the period of service or product consumption.
> * *Seasonality and Trend Analysis:* Easier to model seasonal trends and effects, as the arrival date directly corresponds to the period being analyzed.
> * *Accuracy in Performance Metrics:* Predictions based on arrival date can be more closely aligned with actual occupancy and revenue, potentially improving model accuracy in terms of performance metrics.

### Cons:

> * *Reduced Forecasting Horizon:* The model may be less effective at predicting bookings well in advance since it's oriented around the arrival date. This could limit its usefulness for long-term planning.
> * *Possible Lag in Actionable Insights:* May not provide as much lead time for implementing strategies based on the predictions, such as staffing or promotional offers, since the focus is on the period closer to the actual stay.

---

# Consultation Conclusions

After considering ChatGPT's suggestions and insights, I will take a more greedy approach and create separate datasets for both the booking and arrival dates. This will give me more flexibility when modeling as I will have different time perspectives to utilize for different purposes (e.g., future forecasting vs. analyzing actualized performance).

---

# Date Preparation

Before I can split the datasets, I need to perform some slight feature engineering. The source datasets do not have an exact datetime feature for the arrival date, only for the booking date. I will use the separate Year, Month, and Day of Month features to create an `Arrival_Date` feature, then use this feature for splitting my data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bmc_functions import db_utils
import pandas as pd

In [3]:
### OLD DATA - Retaining for debugging

# ## Maintaing separate hotel data

# hotel_number = '1'
# # hotel_number = '2'

# path = f'./data/H{hotel_number}.parquet'
# df_data = pd.read_parquet(path)

# df_data.head()

# Read Data from DuckDB

In [4]:
# Path to the DuckDB database file
db_path = './data/hotel_reservations.duckdb'

## Select subset of data for review
q = 'SELECT * FROM res_data LIMIT 10'

with db_utils.duckdb_connection(db_path) as conn:
    display(conn.execute(q).df())

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,HotelNumber,UUID
0,0,342,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,6f4f201b-62de-4c33-b9da-15081cf7e359
1,0,737,2015,July,27,1,0,0,2,0,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,1,ad3d542c-41eb-4957-9724-32d3638d6bec
2,0,7,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,6a0684e0-9157-4456-9a53-1685d506a951
3,0,13,2015,July,27,1,0,1,1,0,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,1,512c5081-3be1-4f78-87dc-2b0c8a9e9bde
4,0,14,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,444ea515-6e6b-45b4-9870-fcf46b5b529c
5,0,14,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,1,5c55ead3-f96e-44b6-b167-f0f0c87e013c
6,0,0,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,107.0,0,0,Check-Out,2015-07-03,1,ba6dccfe-4082-4ce8-86dd-9fc130c84d3c
7,0,9,2015,July,27,1,0,2,2,0,...,NULL,0,Transient,103.0,0,1,Check-Out,2015-07-03,1,ea4d5557-6767-458a-8f3d-929cea401bc5
8,1,85,2015,July,27,1,0,3,2,0,...,NULL,0,Transient,82.0,0,1,Canceled,2015-05-06,1,d5ef1deb-f28a-45bb-b7bc-aac85dad3425
9,1,75,2015,July,27,1,0,3,2,0,...,NULL,0,Transient,105.5,0,0,Canceled,2015-04-22,1,f07c2799-6be5-4a8d-93a5-ee099cfa3e3c


In [5]:
## Convert Arrival columns to strings

q = ('''
SELECT uuid, ArrivalDateYear, ArrivalDateMonth, ArrivalDateDayOfMonth,
StaysInWeekNights, StaysInWeekendNights, LeadTime 
FROM res_data''')

with db_utils.duckdb_connection(db_path) as conn:
    df_data = conn.execute(q).df()

# df_data = arrival_cols.astype(str)
df_data.head()

,UUID,ArrivalDateYear,ArrivalDateMonth,ArrivalDateDayOfMonth,StaysInWeekNights,StaysInWeekendNights,LeadTime
0,6f4f201b-62de-4c33-b9da-15081cf7e359,2015,July,1,0,0,342
1,ad3d542c-41eb-4957-9724-32d3638d6bec,2015,July,1,0,0,737
2,6a0684e0-9157-4456-9a53-1685d506a951,2015,July,1,1,0,7
3,512c5081-3be1-4f78-87dc-2b0c8a9e9bde,2015,July,1,1,0,13
4,444ea515-6e6b-45b4-9870-fcf46b5b529c,2015,July,1,2,0,14


# Feature Engineering: Arrival, Departure, and Booking Dates

## Arrival Date

In [6]:
## Create new column of strings formatted as YYYY-MM-DD, then convert to datetime

arrival_details = ['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth']

df_data[arrival_details] = df_data[arrival_details].astype(str)

df_data['ArrivalDate'] = df_data['ArrivalDateYear'].str.cat(df_data[['ArrivalDateMonth',
                                                                     'ArrivalDateDayOfMonth']],
                                                            '-')

df_data['ArrivalDate'] = pd.to_datetime(df_data['ArrivalDate'], yearfirst = True)

df_data.head()

,UUID,ArrivalDateYear,ArrivalDateMonth,ArrivalDateDayOfMonth,StaysInWeekNights,StaysInWeekendNights,LeadTime,ArrivalDate
0,6f4f201b-62de-4c33-b9da-15081cf7e359,2015,July,1,0,0,342,2015-07-01
1,ad3d542c-41eb-4957-9724-32d3638d6bec,2015,July,1,0,0,737,2015-07-01
2,6a0684e0-9157-4456-9a53-1685d506a951,2015,July,1,1,0,7,2015-07-01
3,512c5081-3be1-4f78-87dc-2b0c8a9e9bde,2015,July,1,1,0,13,2015-07-01
4,444ea515-6e6b-45b4-9870-fcf46b5b529c,2015,July,1,2,0,14,2015-07-01


## Departure Date

In [7]:
timedelta_wknd = pd.to_timedelta(df_data.loc[:, 'StaysInWeekendNights'], unit = 'D')
timedelta_wk = pd.to_timedelta(df_data.loc[:, 'StaysInWeekNights'], unit = 'D')

df_data['DepartureDate'] = df_data.loc[:, 'ArrivalDate'] + timedelta_wk + timedelta_wknd

df_data.head()

,UUID,ArrivalDateYear,ArrivalDateMonth,ArrivalDateDayOfMonth,StaysInWeekNights,StaysInWeekendNights,LeadTime,ArrivalDate,DepartureDate
0,6f4f201b-62de-4c33-b9da-15081cf7e359,2015,July,1,0,0,342,2015-07-01,2015-07-01
1,ad3d542c-41eb-4957-9724-32d3638d6bec,2015,July,1,0,0,737,2015-07-01,2015-07-01
2,6a0684e0-9157-4456-9a53-1685d506a951,2015,July,1,1,0,7,2015-07-01,2015-07-02
3,512c5081-3be1-4f78-87dc-2b0c8a9e9bde,2015,July,1,1,0,13,2015-07-01,2015-07-02
4,444ea515-6e6b-45b4-9870-fcf46b5b529c,2015,July,1,2,0,14,2015-07-01,2015-07-03


## Booking Date

In [8]:
df_data['LeadTime']

0         342
1         737
2           7
3          13
4          14
         ... 
119385     23
119386    102
119387     34
119388    109
119389    205
Name: LeadTime, Length: 119390, dtype: int64

In [9]:
df_data['LeadTimeDelta'] = pd.to_timedelta(df_data['LeadTime'], unit = 'D')
df_data['LeadTimeDelta']

0        342 days
1        737 days
2          7 days
3         13 days
4         14 days
           ...   
119385    23 days
119386   102 days
119387    34 days
119388   109 days
119389   205 days
Name: LeadTimeDelta, Length: 119390, dtype: timedelta64[ns]

In [ ]:
# df_data['LeadTimeDelta'] = pd.to_timedelta(df_data['LeadTime'], unit = 'D')

df_data['BookingDate'] = df_data['ArrivalDate'] - df_data

# df_data.head(10)

# Subset with 10% Threshold for Arrival_Date and Booking_Date

Despite the sizable number of rows for each hotel's dataset, the Arrival_Date feature is limited in scope - covering a little over 2 years' worth of data. This limited time range restricts the possible forecasting windows, particularly as I intend to reserve a certain number of days for the validation sets.

My compromise between availability and integrity is to subset each hotel's separate datasets by 10% of the total dataset. This will preserve the size of the original data's timeframe while providing a reasonable time window for future forecasting.

In [ ]:
# def split_and_save_dataset_by_percentage(df, date_column, hotel_number, percentage=10, save_path='./data'):
#     """
#     Split the dataset into training and holdout sets based on a specified percentage of the total rows
#     and save them. The holdout set will contain the last 'percentage' of rows based on the datetime column.
#     """
    
#     # Work on a copy of the DataFrame to preserve the original data
#     df_copy = df.copy()
    
#     # Ensure the date column is in datetime format and set it as the index
#     df_copy[date_column] = pd.to_datetime(df_copy[date_column])
#     df_copy = df_copy.sort_values(by=[date_column])

#     # Calculate the index to split on: last 'percentage' of rows
#     split_idx = int(len(df_copy) * (1 - (percentage / 100)))

#     # Split the dataset and reset index for future use
#     train_df = df_copy.iloc[:split_idx, :]
#     train_df = train_df.reset_index(drop=True)
    
#     holdout_df = df_copy.iloc[split_idx:, :]
#     holdout_df = holdout_df.reset_index(drop=True)

#     # No need to reset index if you want to keep the datetime column in its place
#     # Define file paths
#     training_path = f'{save_path}/H{hotel_number}_Training.parquet'
#     holdout_path = f'{save_path}/H{hotel_number}_Validation.parquet'

#     # Save the datasets
#     train_df.to_parquet(training_path, engine='pyarrow', compression='brotli')
#     holdout_df.to_parquet(holdout_path, engine='pyarrow', compression='brotli')


In [ ]:
# date_columns = ['Arrival_Date', 'Booking_Date']

# for date_column in date_columns:
#     split_and_save_dataset_by_percentage(df_data, date_column, hotel_number, 10, f'./data/Datasets_for_{date_column}')


# Results

After running this code for both of the hotel datasets, I now have separate training and validation datasets for each of the hotels' Arrival_Date and Booking_Date features. The resulting datasets will be effective for modeling and forecasting in later notebooks.

In [ ]:
# ## Create timedelta series based on number of weekday/end nights.
# timedelta_wknd = pd.to_timedelta(df_data.loc[:, 'StaysInWeekendNights'], unit = 'D')
# timedelta_wk = pd.to_timedelta(df_data.loc[:, 'StaysInWeekNights'], unit = 'D')

In [ ]:
# table_name = 'res_data'

# with db_utils.duckdb_connection(db_path) as conn:
#     column_info = conn.execute(f"PRAGMA table_info({table_name})").fetchall()

#     # Print information about each column
#     for column in column_info:
#         column_name, column_dtype = column[1], column[2]
#         print(f"Column Name: {column_name}, Data Type: {column_dtype}")

# _ = db_utils.get_col_dtypes('res_data', db_path)

In [ ]:
# create_dates_table = '''CREATE TABLE reservation_dates (
#     UUID VARCHAR,
#     datetime_column TIMESTAMP,
#     PRIMARY KEY (UUID),
#     FOREIGN KEY (UUID) REFERENCES res_data (UUID)
# );
# '''
# create_dates_table

In [ ]:
# db_utils.alter_table(create_dates_table, db_path)

In [ ]:
# db_utils.alter_table('ALTER TABLE res_data ADD PRIMARY KEY (UUID);', db_path)

In [ ]:
# alter_commands = "ALTER TABLE res_data ADD COLUMN Arrival_Date DATE"
# with duckdb_connection(db_path) as conn:
#     conn.execute(alter_commands)
#     print('Completed successfully.')

# drop_col = 'ALTER TABLE res_data DROP COLUMN Arrival_Date;'
# db_utils.alter_table(drop_col, db_path)

# alt_tbl_cmd = "ALTER TABLE res_data ADD COLUMN Arrival_Date TIMESTAMP"
# db_utils.alter_table(alt_tbl_cmd, db_path)

In [ ]:
# q = 'SELECT Arrival_Date FROM res_data'
# with db_utils.duckdb_connection(db_path) as conn:
#     display(conn.execute(q).df())

In [ ]:
# q = 'SELECT Arrival_Date FROM res_data'
# with db_utils.duckdb_connection(db_path) as conn:
#     result = conn.execute(q).fetchall()

#     for row in result:
#         print(row)

In [ ]:
# ## Concatenate new column
# df_data = pd.concat([df_data, arrival_date_dt], axis = 1)
# df_data.head()

In [ ]:
# try:
#     df_data = df_data.drop(columns=[ 'ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth'])
# except:
#     pass

# df_data.head()

In [ ]:
# q = 'SELECT UUID, StaysInWeekNights, StaysInWeekendNights FROM res_data'

# with db_utils.duckdb_connection(db_path) as conn:
#     stays_cols = conn.execute(q).df()
    
# stays_cols.head()

In [ ]:
# # pd.concat([arrival_date_cols_str, stays_cols], axis = 1)

# df_data = arrival_date_cols_str.merge(stays_cols, how = 'left', on = 'UUID')
# df_data.head()

In [ ]:


## Calculate the departure date by adding the timedeltas to the arrival date
# departure_date = df_data.loc[:, 'ArrivalDate'] + timedelta_wk + timedelta_wknd
# departure_date.name = 'DepartureDate'
# departure_date.head()

# df_data['DepartureDate'] = df_data.loc[:, 'ArrivalDate'] + timedelta_wk + timedelta_wknd
# df_data.head()

In [ ]:
# try:
#     df_data = df_data.drop(columns=['StaysInWeekendNights', 'StaysInWeekNights'])
# except:
#     pass

# df_data.head()

In [ ]:
# q = 'SELECT LeadTime from res_data'

# with db_utils.duckdb_connection(db_path) as conn:
#     leadtime = conn.execute(q).df()
#     print('Completed successfully.')

In [ ]:
# df_data = pd.concat([df_data, leadtime['LeadTime']], axis = 1)

# df_data['LeadTimeDelta'] = pd.to_timedelta(df_data['LeadTime'], unit = 'D')
# df_data.head()

In [ ]:
# df_data['BookingDate'] = df_data['ArrivalDate'] - df_data
# df_data.head(10)

In [ ]:
# df_data = df_data.drop(columns = ['LeadTime']) ## Maintain for future modeling

# df_data.head(10)

In [ ]:
# df_data['Arrival_Date'].max() - df_data['Arrival_Date'].min()

In [ ]:
# df_data['Booking_Date'].max() - df_data['Booking_Date'].min()